<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 6. Redes Neuronales Recurrentes</font></h1>

<h1><font color="#113D68" size=4>4. LSTM para Clasificación</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. LSTM para clasificación](#section1)
    * [1.1. Librerías](#section1.1)
    * [1.2. Dataset](#section1.2)
    * [1.3. Truncar las secuencias de entrada](#section1.3)
    * [1.4. Modelo de linea base](#section1.4)
    * [1.5. Resultados](#section1.5)
* [2. LSTM con Dropout](#section2)
* [3. LSTM con Dropout recurrente](#section3)
* [4. LSTM y CNN para clasificación](#section4)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

La clasificación de secuencias es un problema de modelado predictivo en el que tiene alguna secuencia de entradas en el espacio o el tiempo y la tarea es predecir una categoría para la secuencia. Lo que dificulta este problema es que las secuencias pueden variar en longitud, estar compuestas por un vocabulario muy amplio de símbolos de entrada y pueden requerir que el modelo aprenda el contexto a largo plazo o las dependencias entre símbolos en la secuencia de entrada. En este proyecto, descubrirá cómo puede desarrollar modelos de redes neuronales recurrentes LSTM para problemas de clasificación de secuencias en Python utilizando la biblioteca de aprendizaje profundo de Keras. Después de completar este proyecto, sabrá:

* Cómo desarrollar un modelo LSTM para un problema de clasificación de secuencias.
* Cómo reducir el sobreajuste en sus modelos LSTM mediante el uso de abandonos.
* Cómo combinar modelos LSTM con redes neuronales convolucionales que sobresalen en el aprendizaje de relaciones espaciales.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. LSTM para clasificación</font>

El problema que usaremos para demostrar el aprendizaje de secuencias en este tutorial es el problema de clasificación de sentimientos de revisión de películas de IMDB. Podemos desarrollar rápidamente un pequeño LSTM para el problema de IMDB y lograr una buena precisión. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Puede obtener más información sobre el dataset [IMBD](http://ai.stanford.edu/~amaas/data/sentiment/)


<a id="section1.1"></a>
# <font color="#004D7F" size=5>1.1. Funciones</font>

Comencemos importando las clases y funciones necesarias para este modelo.

In [7]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

<a id="section1.2"></a>
# <font color="#004D7F" size=5>1.2. Dataset</font>

Necesitamos cargar el conjunto de datos de IMDB. Limitamos el conjunto de datos a las 5.000 palabras principales. También dividimos el conjunto de datos en conjuntos de entranmiento (50%) y validación (50%).

In [2]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)


<a id="section1.3"></a>
# <font color="#004D7F" size=5>1.3. Truncar las secuencias de entrada</font>

A continuación, necesitamos truncar y rellenar las secuencias de entrada para que todas tengan la misma longitud para el modelado. El modelo aprenderá que los valores cero no contienen información, de modo que las secuencias no tienen la misma longitud en términos de contenido, pero se requieren vectores de la misma longitud para realizar el cálculo en Keras.

In [3]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

<a id="section1.4"></a>
# <font color="#004D7F" size=5>1.4. Modelo de linea base</font>

Ahora podemos definir, compilar y ajustar nuestro modelo LSTM. La primera capa es la capa incrustada que utiliza 32 vectores de longitud para representar cada palabra. La siguiente capa es la capa LSTM con 100 unidades de memoria (neuronas inteligentes). 

Finalmente, debido a que este es un problema de clasificación, usamos una capa de salida densa con una sola neurona y una función de activación sigmoidea para hacer predicciones 0 o 1 para las dos clases (buenas y malas) en el problema. Debido a que es un problema de clasificación binaria, la pérdida logarítmica se utiliza como función de pérdida (entropía cruzada binaria en Keras). 

Se utiliza el eficiente algoritmo de optimización de ADAM. El modelo es apto para solo 3 épocas porque rápidamente se adapta al problema. Se utiliza un gran tamaño de lote de 64 revisiones para espaciar las actualizaciones de peso.

In [4]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=3, batch_size=64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
391/391 [==============================] - 119s 305ms/step - loss: 0.4594 - accuracy: 0.7775
Epoch 2/3
391/391 [==============================] - 122s 312ms/step - loss: 0.2992 - accuracy: 0.8828
Epoch 3/3
391/391 [==============================] - 105s 269ms/step - loss: 0.2568 - accuracy: 0.8993


<a id="section1.5"></a>
# <font color="#004D7F" size=5>1.5. Resultados</font>

Una vez ajustado, estimamos el rendimiento del modelo en revisiones no vistas.

In [5]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.85%


Puede ver que este sencillo LSTM con pocos ajustes logra resultados casi de vanguardia en el problema de IMDB. Es importante destacar que esta es una plantilla que puede utilizar para aplicar redes LSTM a sus propios problemas de clasificación de secuencia. 

Ahora, echemos un vistazo a algunas extensiones de este modelo simple que quizás también desee incorporar a sus propios problemas.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. LSTM con Dropout</font>

Las redes neuronales recurrentes como LSTM generalmente tienen el problema de sobreajuste. Dropout se puede aplicar entre capas utilizando la capa Dropout. Podemos hacer esto fácilmente agregando nuevas capas Dropout entre las capas Embedding y LSTM y las capas de salida LSTM y Dense. Por ejemplo:

In [8]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout (Dropout)            (None, 500, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
391/391 [==============================] - 97s 247ms/step - loss: 0.5842 - accuracy: 0.7084
Epoch 2/3
39

Podemos ver que Dropout tiene el impacto deseado en el entrenamiento con una tendencia de convergencia ligeramente más lenta y, en este caso, una precisión final menor. 

El modelo probablemente podría usar algunas épocas más de entrenamiento y puede lograr una habilidad más alta (pruébelo y vea). 

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. LSTM con Dropout recurrente</font>

Alternativamente, Dropout se puede aplicar a las conexiones de entrada y recurrentes de las unidades de memoria con el LSTM de manera precisa y por separado. Keras proporciona esta capacidad con parámetros en la capa LSTM, Dropout para configurar el Dropout de entrada y `recurrent_dropout` para configurar el Dropout recurrente. Por ejemplo, podemos modificar el primer ejemplo para agregar Dropout a la entrada y conexiones recurrentes de la siguiente manera:

In [ ]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Podemos ver que Dropout recurrente de LSTM tiene un efecto más pronunciado en la convergencia de la red que la Dropout por capas. 

Como se indicó anteriormente, el número de épocas se mantuvo constante y podría aumentarse para ver si la habilidad del modelo se puede mejorar aún más. 

Dropout es una técnica poderosa para combatir el sobreajuste en sus modelos LSTM y es una buena idea probar ambos métodos, pero puede apostar mejores resultados con la deserción específica de la puerta proporcionada en Keras.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section4"></a>
# <font color="#004D7F" size=6>4. LSTM y CNN para clasificación</font>

Las redes neuronales convolucionales sobresalen en el aprendizaje de la estructura espacial en los datos de entrada. Los datos de revisión de IMDB tienen una estructura espacial unidimensional en la secuencia de palabras en las revisiones y la CNN puede seleccionar características invariantes para el sentimiento bueno y malo. Estas características espaciales aprendidas pueden luego ser aprendidas como secuencias por una capa LSTM. Podemos agregar fácilmente una CNN unidimensional y capas de agrupación máxima después de la capa de incrustación que luego alimentan las características consolidadas al LSTM.

Podemos utilizar un conjunto más pequeño de 32 entidades con una longitud de filtro pequeña de 3. La capa de agrupación puede utilizar la longitud estándar de 2 para reducir a la mitad el tamaño del mapa de entidades. Por ejemplo, crearíamos el modelo de la siguiente manera:

In [ ]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Podemos ver que logramos resultados similares al primer ejemplo aunque con menos pesos y un tiempo de entrenamiento más rápido. Esperaría que se pudieran lograr resultados aún mejores si este ejemplo se ampliara aún más para usar el abandono.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>